In [2]:
import os
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'

from langchain import PromptTemplate, FewShotPromptTemplate, LLMChain, OpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

parser = StrOutputParser()

model = ChatOpenAI(model="gpt-4o-mini")

In [2]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

examples = [
    {"input": "2 🦜 2", "output": "4"},
    {"input": "2 🦜 3", "output": "5"},
]

In [84]:
# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    # suffix="Human: {input}",
)



[HumanMessage(content='2 🦜 2'), AIMessage(content='4'), HumanMessage(content='2 🦜 3'), AIMessage(content='5')]


In [93]:
few_shot_prompt.invoke({}).messages

[HumanMessage(content='2 🦜 2'),
 AIMessage(content='4'),
 HumanMessage(content='2 🦜 3'),
 AIMessage(content='5')]

In [86]:
few_shot_prompt.format()

'Human: 2 🦜 2\nAI: 4\nHuman: 2 🦜 3\nAI: 5'

In [ ]:
print(few_shot_prompt.invoke({}).to_messages())

In [94]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)
# chain = model | final_prompt
chain = final_prompt | model

chain.invoke({"input": "What's 3 🦜 3?"})

AIMessage(content='Based on the previous pattern, the 🦜 operation appears to be addition. Therefore:\n\n\\[ 3 🦜 3 = 3 + 3 = 6 \\]', response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 30, 'total_tokens': 67}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='run-1e976e57-ef4f-4407-9423-2ee339e6352c-0', usage_metadata={'input_tokens': 30, 'output_tokens': 37, 'total_tokens': 67})

In [28]:
final_prompt.format(input="What's 3 🦜 3?")

"System: You are a wondrous wizard of math.\nHuman: 2 🦜 2\nAI: 4\nHuman: 2 🦜 3\nAI: 5\nHuman: What's 3 🦜 3?"

In [59]:
print(final_prompt.format(input="What's 3 🦜 3?"))

System: You are a wondrous wizard of math.
Human: 2 🦜 2
AI: 4
Human: 2 🦜 3
AI: 5
Human: What's 3 🦜 3?


In [3]:
from langchain_core.prompts import PromptTemplate

example_prompt = PromptTemplate.from_template("Question: {question}\n{answer}")

In [4]:
qa_examples = [
    {
        "question": "Who lived longer, Muhammad Ali or Alan Turing?",
        "answer": """
            Are follow up questions needed here: Yes.
            Follow up: How old was Muhammad Ali when he died?
            Intermediate answer: Muhammad Ali was 74 years old when he died.
            Follow up: How old was Alan Turing when he died?
            Intermediate answer: Alan Turing was 41 years old when he died.
            So the final answer is: Muhammad Ali
            """,
    },
    {
        "question": "When was the founder of craigslist born?",
        "answer": """
            Are follow up questions needed here: Yes.
            Follow up: Who was the founder of craigslist?
            Intermediate answer: Craigslist was founded by Craig Newmark.
            Follow up: When was Craig Newmark born?
            Intermediate answer: Craig Newmark was born on December 6, 1952.
            So the final answer is: December 6, 1952
            """,
    },
    {
        "question": "Who was the maternal grandfather of George Washington?",
        "answer": """
            Are follow up questions needed here: Yes.
            Follow up: Who was the mother of George Washington?
            Intermediate answer: The mother of George Washington was Mary Ball Washington.
            Follow up: Who was the father of Mary Ball Washington?
            Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
            So the final answer is: Joseph Ball
            """,
    },
    {
        "question": "Are both the directors of Jaws and Casino Royale from the same country?",
        "answer": """
            Are follow up questions needed here: Yes.
            Follow up: Who is the director of Jaws?
            Intermediate Answer: The director of Jaws is Steven Spielberg.
            Follow up: Where is Steven Spielberg from?
            Intermediate Answer: The United States.
            Follow up: Who is the director of Casino Royale?
            Intermediate Answer: The director of Casino Royale is Martin Campbell.
            Follow up: Where is Martin Campbell from?
            Intermediate Answer: New Zealand.
            So the final answer is: No
            """,
    },
]

In [5]:
print(example_prompt.invoke(qa_examples[0]).text)

Question: Who lived longer, Muhammad Ali or Alan Turing?

            Are follow up questions needed here: Yes.
            Follow up: How old was Muhammad Ali when he died?
            Intermediate answer: Muhammad Ali was 74 years old when he died.
            Follow up: How old was Alan Turing when he died?
            Intermediate answer: Alan Turing was 41 years old when he died.
            So the final answer is: Muhammad Ali
            


In [12]:
from langchain_core.prompts import FewShotPromptTemplate

prompt = FewShotPromptTemplate(
    examples=qa_examples,
    example_prompt=example_prompt,
    # prefix="You are a helpful assistant.",
    suffix="Question: {input}",
    input_variables=["input"],
)

print(
    prompt.invoke({"input": "Who was the father of Mary Ball Washington?"}).to_string()
)

Question: Who lived longer, Muhammad Ali or Alan Turing?

            Are follow up questions needed here: Yes.
            Follow up: How old was Muhammad Ali when he died?
            Intermediate answer: Muhammad Ali was 74 years old when he died.
            Follow up: How old was Alan Turing when he died?
            Intermediate answer: Alan Turing was 41 years old when he died.
            So the final answer is: Muhammad Ali
            

Question: When was the founder of craigslist born?

            Are follow up questions needed here: Yes.
            Follow up: Who was the founder of craigslist?
            Intermediate answer: Craigslist was founded by Craig Newmark.
            Follow up: When was Craig Newmark born?
            Intermediate answer: Craig Newmark was born on December 6, 1952.
            So the final answer is: December 6, 1952
            

Question: Who was the maternal grandfather of George Washington?

            Are follow up questions needed here: 

In [13]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    qa_examples,
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(),
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    Chroma,
    # This is the number of examples to produce.
    k=1,
)

In [14]:
# Select the most similar example to the input.
question = "Who was the father of Mary Ball Washington?"
selected_examples = example_selector.select_examples({"question": question})
print(f"Examples most similar to the input: {question}")
for example in selected_examples:
    print("\n")
    print('【')
    for k, v in example.items():
        print(f"{k}: {v}")
    print('】')

Examples most similar to the input: Who was the father of Mary Ball Washington?


【
answer: 
            Are follow up questions needed here: Yes.
            Follow up: Who was the mother of George Washington?
            Intermediate answer: The mother of George Washington was Mary Ball Washington.
            Follow up: Who was the father of Mary Ball Washington?
            Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
            So the final answer is: Joseph Ball
            
question: Who was the maternal grandfather of George Washington?
】


In [15]:
selected_examples

[{'answer': '\n            Are follow up questions needed here: Yes.\n            Follow up: Who was the mother of George Washington?\n            Intermediate answer: The mother of George Washington was Mary Ball Washington.\n            Follow up: Who was the father of Mary Ball Washington?\n            Intermediate answer: The father of Mary Ball Washington was Joseph Ball.\n            So the final answer is: Joseph Ball\n            ',
  'question': 'Who was the maternal grandfather of George Washington?'}]

In [16]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

print(
    prompt.invoke({"input": "Who was the father of Mary Ball Washington?"}).to_string()
)

Question: Who was the maternal grandfather of George Washington?

            Are follow up questions needed here: Yes.
            Follow up: Who was the mother of George Washington?
            Intermediate answer: The mother of George Washington was Mary Ball Washington.
            Follow up: Who was the father of Mary Ball Washington?
            Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
            So the final answer is: Joseph Ball
            

Question: Who was the father of Mary Ball Washington?


In [17]:
chain = prompt | model
chain.invoke({"input": "Who was the father of Mary Ball Washington?"})

AIMessage(content='The father of Mary Ball Washington was Joseph Ball.', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 103, 'total_tokens': 113}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0f03d4f0ee', 'finish_reason': 'stop', 'logprobs': None}, id='run-ae96f9c7-ac89-47ba-8074-69197b89bef5-0', usage_metadata={'input_tokens': 103, 'output_tokens': 10, 'total_tokens': 113})